In [1]:
import tensorflow as tf
import pandas as pd
import keras
from keras import regularizers
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train.head()

,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,aPos,aGSPG,aGCPG,aCSPG,aPtsPG,...,aGD5,aGD7,hPosLS,aPosLS,hSpent,hIncome,hNet,aSpent,aIncome,aNet
0,4.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18,18,18.72,0.479,-18.241,18.07,1.320,-16.750
1,5.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5,9,47.15,34.830,-12.320,8.92,38.160,29.240
2,6.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,14,12,17.10,23.650,6.550,29.52,9.000,-20.520
3,7.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,16,15,11.99,6.260,-5.730,0.00,0.131,0.131
4,10.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,13,47.84,10.920,-36.920,31.03,32.000,0.970


In [4]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 36)


array([ 0.03914121,  0.01673287,  0.00939389,  0.00410983,  0.01673287,
        0.11742364,  0.01262304,  0.01184022,  0.00322915,  0.01262304,
        0.12190531,  0.06401546,  0.10600419,  0.05178383,  0.00081218,
        0.00097853,  0.0097853 ,  0.00782824,  0.0111846 ,  0.00325851,
       -0.00195706,  0.0013993 ,  0.01304381,  0.01957061,  0.01957061,
        0.00325851,  0.00782824,  0.0097853 ,  0.05871182,  0.04892652,
        0.        ,  0.01849422,  0.01849422,  0.03963048,  0.7055204 ,
        0.66588992])

In [5]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [6]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [7]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [8]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
# model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
# model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))

In [9]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
model.fit(X_train_norm, y_train.values, epochs=15, validation_data=(X_test_norm,y_test), class_weight=class_weights)

Train on 4180 samples, validate on 1520 samples
Epoch 1/15
4180/4180 [==============================] - 1s 142us/step - loss: 1.0946 - acc: 0.4411 - val_loss: 1.0827 - val_acc: 0.4474
Epoch 2/15
4180/4180 [==============================] - 0s 76us/step - loss: 1.0843 - acc: 0.4349 - val_loss: 1.0629 - val_acc: 0.4875
Epoch 3/15
4180/4180 [==============================] - 0s 61us/step - loss: 1.0699 - acc: 0.4620 - val_loss: 1.0462 - val_acc: 0.4901
Epoch 4/15
4180/4180 [==============================] - 0s 61us/step - loss: 1.0584 - acc: 0.4651 - val_loss: 1.0344 - val_acc: 0.5000
Epoch 5/15
4180/4180 [==============================] - 0s 67us/step - loss: 1.0516 - acc: 0.4605 - val_loss: 1.0260 - val_acc: 0.5059
Epoch 6/15
4180/4180 [==============================] - 0s 61us/step - loss: 1.0480 - acc: 0.4660 - val_loss: 1.0215 - val_acc: 0.5066
Epoch 7/15
4180/4180 [==============================] - 0s 66us/step - loss: 1.0445 - acc: 0.4627 - val_loss: 1.0169 - val_acc: 0.5132
Epoch 

In [11]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 32us/step
1.012799593021995 0.5078947368421053


In [15]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(preds).value_counts()

2    733
0    555
1    232
dtype: int64

In [17]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 267, 1: 64, 2: 441}

In [14]:
733/1520

0.48223684210526313